In [1]:
!pip install pillow fpdf python-docx


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 8.1 MB/s eta 0:00:00
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=40b4824ef5c4a5dc29dcb7be5b977e29819a0295f3ef94aa178900fbd7d57e6e
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf


In [10]:
from google.colab import files
uploaded = files.upload()


Saving Julia_Handwritten_2[1].ttf to Julia_Handwritten_2[1].ttf


In [33]:
from PIL import Image, ImageDraw, ImageFont, ImageFilter

def generate_custom_handwriting_interactive():
    # Available options
    font_options = {
        "patrick": "/content/PatrickHand-Regular[1].ttf",
        "journal": "/content/JOURNAL[1].TTF",
        "homemade apple": "/content/HomemadeApple-Regular[1].ttf",
        "julia": "/content/Julia_Handwritten_2[1].ttf"
    }

    paper_options = {
        "notebook": "/content/surface-white-planks.jpg",
        "a4": "/content/plain A4 page.jpg"
    }

    ink_colors = {
        "blue": (0, 0, 255, 180),
        "black": (0, 0, 0, 180)
    }

    # --- User Inputs ---
    print("Available Fonts: ", list(font_options.keys()))
    font_choice = input("Select font style: ").strip().lower()
    font_choice = font_choice if font_choice in font_options else "patrick"

    print("Available Papers: ", list(paper_options.keys()))
    paper_choice = input("Select paper type: ").strip().lower()
    paper_choice = paper_choice if paper_choice in paper_options else "notebook"

    print("Available Ink Colors: ", list(ink_colors.keys()))
    ink = input("Select ink color (blue/black): ").strip().lower()
    ink = ink if ink in ink_colors else "blue"

    font_size = input("Font size (e.g., 96 or 110): ")
    font_size = int(font_size) if font_size.isdigit() else 96

    text = input("Enter your text to convert to handwriting:\n")

    # --- Load background and font ---
    background = Image.open(paper_options[paper_choice]).convert("RGB")
    width, height = background.size
    font = ImageFont.truetype(font_options[font_choice], size=font_size)

    # --- Text wrapping based on pixel width ---
    def wrap_text_by_pixel(draw, text, font, max_width):
        lines = []
        current_line = ""
        for word in text.split():
            test_line = current_line + " " + word if current_line else word
            bbox = draw.textbbox((0, 0), test_line, font=font)
            line_width = bbox[2] - bbox[0]
            if line_width <= max_width:
                current_line = test_line
            else:
                lines.append(current_line)
                current_line = word
        if current_line:
            lines.append(current_line)
        return lines

    margin = 80
    max_text_width = width - 2 * margin
    temp_draw = ImageDraw.Draw(background)
    wrapped_lines = wrap_text_by_pixel(temp_draw, text, font, max_text_width)

    # --- Create transparent layer and draw ---
    text_layer = Image.new("RGBA", background.size, (255, 255, 255, 0))
    draw = ImageDraw.Draw(text_layer)

    y = margin
    for line in wrapped_lines:
        draw.text((margin, y), line, fill=ink_colors[ink], font=font)
        y += font_size + 10

    # --- Apply slight blur to mimic ink spreading ---
    blurred_text_layer = text_layer.filter(ImageFilter.GaussianBlur(radius=0.7))

    # --- Combine and save ---
    final = Image.alpha_composite(background.convert("RGBA"), blurred_text_layer)
    final_image = final.convert("RGB")
    final_image.show()
    final_image.save("realistic_handwriting_output.png")
    final_image.save("realistic_handwriting_output.pdf")

    print(" Output saved as image and PDF successfully.")

# 🔁 Run the interactive function
generate_custom_handwriting_interactive()

Available Fonts:  ['patrick', 'journal', 'homemade apple', 'julia']
Select font style: journal
Available Papers:  ['notebook', 'a4']
Select paper type: notebook
Available Ink Colors:  ['blue', 'black']
Select ink color (blue/black): blue
Font size (e.g., 96 or 110): 96
Enter your text to convert to handwriting:
 What I Learned: Most people stop at “Colab accuracy scores”… But real-world ML is more than a report — it's about deployment, usability, and impact. I learned how to:  ✅ Move models from notebook to interface ✅ Make predictions user-friendly & interactive ✅ Build apps people can actually use  🔍 What Makes This Advanced? Instead of relying on a single model, I used:  ✅ Logistic Regression for interpretability & benchmarking ✅ Random Forest for powerful decision-making  💡 This dual-model strategy helped me balance accuracy with explainability — a crucial trade-off in real applications.  📦 Key Features: Complete data pipeline (cleaning, scaling, encoding)  Insightful EDA with real

In [54]:
from google.colab import files

uploaded = files.upload()


In [59]:
import os
os.listdir("/content")


['.config',
 'HomemadeApple-Regular[1].ttf',
 '.ipynb_checkpoints',
 'A4page.jpg',
 'Julia_Handwritten_2[1].ttf',
 'PatrickHand-Regular[1].ttf',
 '.gradio',
 'JOURNAL[1].TTF',
 'notebook.jpg',
 'sample_data']

In [67]:
# app.py

import gradio as gr
from PIL import Image, ImageDraw, ImageFont, ImageFilter, ImageColor
import textwrap
import os
import tempfile

FONT_OPTIONS = {
    "Patrick Hand": "PatrickHand-Regular.ttf",
    "Journal": "JOURNAL.ttf",
    "Homemade Apple": "HomemadeApple-Regular.ttf",
    "Julia": "Julia_Handwritten.ttf"
}

PAPER_OPTIONS = {
    "notebook": "notebook.jpg",
    "a4": "A4page.jpg"
}

def generate_handwriting(text, font_choice, uploaded_font, font_mode, page_type, font_size, ink_color):
    try:
        bg_path = PAPER_OPTIONS.get(page_type)
        if not os.path.exists(bg_path):
            return None, None, f"Background image not found at {bg_path}"
        background = Image.open(bg_path).convert("RGB")
        width, height = background.size

        if font_mode == "Use built-in font":
            font_path = FONT_OPTIONS.get(font_choice)
        elif uploaded_font is not None:
            font_path = uploaded_font.name
        else:
            return None, None, "No font selected or uploaded."

        if not os.path.exists(font_path):
            return None, None, f"Font file not found at {font_path}"

        font = ImageFont.truetype(font_path, size=font_size)

        margin = 100
        max_width = width - 2 * margin
        draw = ImageDraw.Draw(background)
        lines = textwrap.wrap(text, width=50)
        y_text = margin

        text_layer = Image.new("RGBA", background.size, (255, 255, 255, 0))
        draw_layer = ImageDraw.Draw(text_layer)

        r, g, b = ImageColor.getrgb(ink_color)
        ink_with_alpha = (r, g, b, 160)

        for line in lines:
            draw_layer.text((margin, y_text), line, font=font, fill=ink_with_alpha)
            y_text += font_size + 15

        blurred_layer = text_layer.filter(ImageFilter.GaussianBlur(radius=0.6))
        final_image = Image.alpha_composite(background.convert("RGBA"), blurred_layer).convert("RGB")

        image_file = tempfile.NamedTemporaryFile(delete=False, suffix=".png")
        final_image.save(image_file.name)

        pdf_file = tempfile.NamedTemporaryFile(delete=False, suffix=".pdf")
        final_image.save(pdf_file.name, "PDF", resolution=100.0)

        return image_file.name, pdf_file.name, None

    except Exception as e:
        return None, None, f"Error: {str(e)}"

with gr.Blocks() as demo:
    gr.HTML("<h1 style='text-align: center;'>AUTO HAND - Handwritten Text Generator</h1>")

    with gr.Accordion("ℹ️ How to Upload Your Handwriting (.ttf)", open=False):
        gr.Markdown("""
### Convert Your Own Handwriting to a Font File (.TTF)

You can make this app use your own handwriting! Just follow these steps:

1. Visit [www.calligraphr.com](https://www.calligraphr.com)
2. Sign up (free plan is fine)
3. Download the handwriting template.
4. Fill it using your own handwriting and a **blue/black pen**.
5. Scan or take a clear photo and upload it back to Calligraphr.
6. Download the generated `.ttf` file.
7. Upload that `.ttf` here in the app to convert any text into **your handwriting!**
""")

    text = gr.Textbox(label="Enter your text", lines=4)
    font_mode = gr.Radio(["Use built-in font", "Upload your own handwriting (.ttf)"], label="Font Mode")
    font_choice = gr.Dropdown(choices=list(FONT_OPTIONS.keys()), label="Choose a built-in font", visible=True)
    uploaded_font = gr.File(label="Upload your .ttf file", file_types=[".ttf"], visible=False)

    def toggle_fonts(choice):
        return {
            font_choice: gr.update(visible=(choice == "Use built-in font")),
            uploaded_font: gr.update(visible=(choice == "Upload your own handwriting (.ttf)"))
        }

    font_mode.change(fn=toggle_fonts, inputs=font_mode, outputs=[font_choice, uploaded_font])

    page_type = gr.Radio(["notebook", "a4"], label="Page Type")
    font_size = gr.Slider(20, 110, value=48, label="Font Size")
    ink_color = gr.ColorPicker(label="Ink Color", value="#000000")

    output_image = gr.Image(label="Preview", type="filepath")
    download_png = gr.File(label="Download PNG", visible=True)
    download_pdf = gr.File(label="Download PDF", visible=True)
    error_box = gr.Textbox(label="Error Message", lines=2, visible=True)

    def generate_and_return_file(*args):
        image_path, pdf_path, error = generate_handwriting(*args)
        if image_path:
            return image_path, image_path, pdf_path, ""
        else:
            return None, None, None, error

    gr.Button("Generate").click(
        fn=generate_and_return_file,
        inputs=[text, font_choice, uploaded_font, font_mode, page_type, font_size, ink_color],
        outputs=[output_image, download_png, download_pdf, error_box]
    )

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://065aaaf04abbb22602.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [68]:
code = '''<# app.py

import gradio as gr
from PIL import Image, ImageDraw, ImageFont, ImageFilter, ImageColor
import textwrap
import os
import tempfile

FONT_OPTIONS = {
    "Patrick Hand": "PatrickHand-Regular.ttf",
    "Journal": "JOURNAL.ttf",
    "Homemade Apple": "HomemadeApple-Regular.ttf",
    "Julia": "Julia_Handwritten.ttf"
}

PAPER_OPTIONS = {
    "notebook": "notebook.jpg",
    "a4": "A4page.jpg"
}

def generate_handwriting(text, font_choice, uploaded_font, font_mode, page_type, font_size, ink_color):
    try:
        bg_path = PAPER_OPTIONS.get(page_type)
        if not os.path.exists(bg_path):
            return None, None, f"Background image not found at {bg_path}"
        background = Image.open(bg_path).convert("RGB")
        width, height = background.size

        if font_mode == "Use built-in font":
            font_path = FONT_OPTIONS.get(font_choice)
        elif uploaded_font is not None:
            font_path = uploaded_font.name
        else:
            return None, None, "No font selected or uploaded."

        if not os.path.exists(font_path):
            return None, None, f"Font file not found at {font_path}"

        font = ImageFont.truetype(font_path, size=font_size)

        margin = 100
        max_width = width - 2 * margin
        draw = ImageDraw.Draw(background)
        lines = textwrap.wrap(text, width=50)
        y_text = margin

        text_layer = Image.new("RGBA", background.size, (255, 255, 255, 0))
        draw_layer = ImageDraw.Draw(text_layer)

        r, g, b = ImageColor.getrgb(ink_color)
        ink_with_alpha = (r, g, b, 160)

        for line in lines:
            draw_layer.text((margin, y_text), line, font=font, fill=ink_with_alpha)
            y_text += font_size + 15

        blurred_layer = text_layer.filter(ImageFilter.GaussianBlur(radius=0.6))
        final_image = Image.alpha_composite(background.convert("RGBA"), blurred_layer).convert("RGB")

        image_file = tempfile.NamedTemporaryFile(delete=False, suffix=".png")
        final_image.save(image_file.name)

        pdf_file = tempfile.NamedTemporaryFile(delete=False, suffix=".pdf")
        final_image.save(pdf_file.name, "PDF", resolution=100.0)

        return image_file.name, pdf_file.name, None

    except Exception as e:
        return None, None, f"Error: {str(e)}"

with gr.Blocks() as demo:
    gr.HTML("<h1 style='text-align: center;'>AUTO HAND - Handwritten Text Generator</h1>")

    with gr.Accordion("ℹ️ How to Upload Your Handwriting (.ttf)", open=False):
        gr.Markdown("""
### Convert Your Own Handwriting to a Font File (.TTF)

You can make this app use your own handwriting! Just follow these steps:

1. Visit [www.calligraphr.com](https://www.calligraphr.com)
2. Sign up (free plan is fine)
3. Download the handwriting template.
4. Fill it using your own handwriting and a **blue/black pen**.
5. Scan or take a clear photo and upload it back to Calligraphr.
6. Download the generated `.ttf` file.
7. Upload that `.ttf` here in the app to convert any text into **your handwriting!**
""")

    text = gr.Textbox(label="Enter your text", lines=4)
    font_mode = gr.Radio(["Use built-in font", "Upload your own handwriting (.ttf)"], label="Font Mode")
    font_choice = gr.Dropdown(choices=list(FONT_OPTIONS.keys()), label="Choose a built-in font", visible=True)
    uploaded_font = gr.File(label="Upload your .ttf file", file_types=[".ttf"], visible=False)

    def toggle_fonts(choice):
        return {
            font_choice: gr.update(visible=(choice == "Use built-in font")),
            uploaded_font: gr.update(visible=(choice == "Upload your own handwriting (.ttf)"))
        }

    font_mode.change(fn=toggle_fonts, inputs=font_mode, outputs=[font_choice, uploaded_font])

    page_type = gr.Radio(["notebook", "a4"], label="Page Type")
    font_size = gr.Slider(20, 110, value=48, label="Font Size")
    ink_color = gr.ColorPicker(label="Ink Color", value="#000000")

    output_image = gr.Image(label="Preview", type="filepath")
    download_png = gr.File(label="Download PNG", visible=True)
    download_pdf = gr.File(label="Download PDF", visible=True)
    error_box = gr.Textbox(label="Error Message", lines=2, visible=True)

    def generate_and_return_file(*args):
        image_path, pdf_path, error = generate_handwriting(*args)
        if image_path:
            return image_path, image_path, pdf_path, ""
        else:
            return None, None, None, error

    gr.Button("Generate").click(
        fn=generate_and_return_file,
        inputs=[text, font_choice, uploaded_font, font_mode, page_type, font_size, ink_color],
        outputs=[output_image, download_png, download_pdf, error_box]
    )

demo.launch()
>'''

with open("app.py", "w") as f:
    f.write(code)

